# Preliminary Imports

In [57]:
import pandas as pd
import numpy as np
from random import randrange
from functools import reduce
from tail_recursive import tail_recursive

# Checking Out Some Statistics of the Data Set

In [18]:
df = pd.read_csv("childcare_costs.csv")
df.index.to_series().map(lambda i: df.iloc[i].isna().sum() / len(df.iloc[i])).to_csv("nancount.csv")

In [20]:
count = 0
for col in df.columns:
    count += df[col].isna().sum()
print(count / (df.shape[0] * df.shape[1]))
    

0.042035732933950555


Grouping the attributes by datatype, we can count the number of attributes for each datatype.

In [ ]:
pd.DataFrame(df.dtypes, columns = ["Type"]).groupby(by=["Type"])['Type'].count()

# k-Means Clustering Algorithm

## Preliminaries

I am using a small data set to test on. This data corresponds to soybean data.

In [3]:
df = pd.read_csv("soybean_data.csv", index_col=0)

## Create Distance Matrix Function

In [8]:
def distMatrix(df, centroids):
    df_mat, cent_mat = df.to_numpy(), centroids.to_numpy()
    return np.array([[np.linalg.norm(df_mat[i, :] - cent_mat[j, :]) for j in range(cent_mat.shape[0])] for i in range(df_mat.shape[0])])

## Assign New Clusters

In [31]:
def cluster(dm):
    def f(i):
        return reduce(lambda x, y: x if x[0] <= y[0] else y, zip(dm[i, :], range(len(dm[i, :]))))[1]
    return f

## Bringing It All Together

In [66]:
def kmeans(df, k, eps):
    @tail_recursive
    def go(currentClus, prevClus, r):
        print("Iteration Number {}".format(r))
        centroids = pd.DataFrame(df.to_dict(orient='series') | {"Cluster": currentClus}).groupby(by=["Cluster"]).mean()
        if prevClus is not None and ((currentClus != prevClus).sum() / len(currentClus)) < eps:
            return centroids
        else:
            dm = distMatrix(df, centroids)
            return go.tail_call(df.index.map(cluster(dm)), currentClus, r + 1)
    return go([randrange(k) for i in range(df.shape[0])], None, 0)

In [67]:
kmeans(df, 4, 0.001)

,Date,Plant-Stand,Precip,Temp,Hail,Crop-Hist,Area-Damaged,Severity,Seed-TMT,Germination,...,Int-Discolor,Sclerotia,Fruit-Pods,Fruit Spots,Seed,Mold-Growth,Seed-Discolor,Seed-Size,Shriveling,Roots
Cluster,,,,,,,,,,,,,,,,,,,,,
0,3.000000,0.333333,2.000000,0.333333,1.000000,1.333333,1.000000,2.000000,0.000000,1.666667,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.333333
1,1.083333,1.000000,1.833333,0.375000,0.208333,1.750000,1.083333,1.541667,0.541667,1.083333,...,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.708333
2,4.500000,0.000000,2.000000,1.000000,0.100000,1.900000,0.300000,1.300000,0.500000,1.300000,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,4.700000,0.000000,0.000000,1.600000,0.600000,1.600000,2.500000,1.000000,0.500000,0.900000,...,2.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.000000
